In [1]:
"""라이브러리 불러오기"""
import pandas as pd #데이터를 처리하는 라이브러리인 pandas를 불러옴
import numpy as np #연산을 위한 numpy 라이브러리를 불러옴
from sklearn.linear_model import LinearRegression #sklearn 라이브러리에서 선형회귀(Linear Regression) 모델 임포트
from sklearn.model_selection import train_test_split #sklearn 라이브러리에서 train, test set를 분류하는 함수 임포트
from sklearn.metrics import mean_absolute_error #sklearn 라이브러리에서 평균 절대 오차를 계산하는 함수 임포트

"""데이터 불러오기"""
train = pd.read_csv("train.csv")
test = pd.read_csv("test.csv")
submission = pd.read_csv("sample_submission.csv")

print(train.info())
print(train.head())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 273 entries, 0 to 272
Data columns (total 10 columns):
 #   Column                     Non-Null Count  Dtype  
---  ------                     --------------  -----  
 0   date_time                  273 non-null    object 
 1   wind_direction             273 non-null    float64
 2   sky_condition              273 non-null    float64
 3   precipitation_form         273 non-null    float64
 4   wind_speed                 273 non-null    float64
 5   humidity                   273 non-null    float64
 6   low_temp                   273 non-null    float64
 7   high_temp                  273 non-null    float64
 8   Precipitation_Probability  273 non-null    float64
 9   number_of_rentals          273 non-null    int64  
dtypes: float64(8), int64(1), object(1)
memory usage: 21.5+ KB
None
    date_time  wind_direction  sky_condition  precipitation_form  wind_speed  \
0  2018-04-01         207.500          4.000               0.000       3.05

In [2]:
"""데이터 전처리"""
X = train.drop(columns=["number_of_rentals", "date_time"])
#학습 데이터에서 예측값인 number_of_rentals와 영향을 주지 않는 date_time을 학습대상에서 제외, 나머지를 X로 지정

y = train["number_of_rentals"] #target 지정
X_test = test.drop(columns=["date_time"]) #테스트 데이터에서 date_time을 제외한 나머지를 X_test로 지정

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.2, random_state=42)
#교차검증을 위해 train 데이터를 train과 valid로 나눔

"""훈련 및 예측"""
model = LinearRegression() #선형회귀 모델 사용
model.fit(X_train, y_train)

train_pred = model.predict(X_train) 
valid_pred = model.predict(X_valid)
test_pred = model.predict(X_test)
#예측한 값을 저장함.


"""평균절대오차 계산 (평가지표-NMAE)"""

def NMAE(true, pred):
    return np.mean(np.abs(true - pred)) / np.mean(true)
    #평균 절대 오차를 계산하는 함수 NMAE 정의 (평균 절대 오차 / 평균값)

nmae_train = NMAE(y_train, train_pred) #train 데이터에 대한 평균절대오차 계산
nmae_valid = NMAE(y_valid, valid_pred) #valid 데이터에 대한 평균절대오차 계산
print(f"모델 Train NMAE: {nmae_train}") #train 데이터에 대한 평균절대오차 출력
print(f"모델 Validation NMAE: {nmae_valid}") #valid 데이터에 대한 평균절대오차 출력

"""baseline에서는 직접 함수를 만들었지만 위에서 임포트한 mean_absolute_error 함수를 사용하여 평균 절대 오차 계산할 수도 있음"""
# mae_train = mean_absolute_error(y_train, train_pred)
# mae_valid = mean_absolute_error(y_valid, valid_pred)
# print(f"모델 Train MAE: {mae_train}")
# print(f"모델 Validation MAE: {mae_valid}")
#nmae_train = mean_absolute_error(y_train, train_pred) / np.mean(y_train) #train 데이터에 대한 평균절대오차 계산

"""제출 파일 생성"""

submission["number_of_rentals"] = test_pred #예측한 값(대여수) 삽입
submission.to_csv("baseline_submission.csv", index=False) #파일 저장, idx(행번호)는 저장하지 않음
print("Baseline submission file saved.")

모델 Train NMAE: 0.3051150098891526
모델 Validation NMAE: 0.30211693027453346
Baseline submission file saved.


In [3]:
#민수님 코드

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_absolute_error

LR_x = train.drop(columns=["number_of_rentals","date_time"])
LR_y = train["number_of_rentals"]
x_test = test.drop(columns=["date_time"])

LR_x_train,LR_x_valid, LR_y_train, LR_y_valid = train_test_split(LR_x, LR_y, test_size=0.2, random_state=42)
model = LinearRegression()
model.fit(LR_x_train,LR_y_train)

LR_train_pred = model.predict(LR_x_train)
LR_valid_pred = model.predict(LR_x_valid)

LR_nmae_train = NMAE(LR_y_train, LR_train_pred)
LR_nmae_valid = NMAE(LR_y_valid, LR_valid_pred)

#final_pred = model.predict(x_test)
#submission["number_of_rentals"] = final_pred
#submission.to_csv("submission.csv", index=False)

# print("🚲 데이콘 제출용 파일 저장 완료! → 'submission.csv'")
# check_df = pd.read_csv("submission.csv")
# print(check_df.head(10))
print(f"LinearRegressor 훈련 NMAE:{LR_nmae_train}")
print(f"LinearRegressor 검증 NMAE : {LR_nmae_valid}")

# coefficients = model.coef_
# features = LR_x_train.columns
# coef_df = pd.Series(np.abs(coefficients), index=features)
# coef_df.sort_values().plot(kind='barh')
# plt.title("Linear Regression Coefficient Importance")
# plt.show()

LinearRegressor 훈련 NMAE:0.3051150098891526
LinearRegressor 검증 NMAE : 0.30211693027453346
